In [1]:
pip install opencv-python-headless numpy open3d

   ---------------------------------------- 0.0/63.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/63.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/63.3 MB ? eta -:--:--
    --------------------------------------- 1.3/63.3 MB 3.2 MB/s eta 0:00:20
   - -------------------------------------- 2.6/63.3 MB 4.2 MB/s eta 0:00:15
   -- ------------------------------------- 3.9/63.3 MB 5.0 MB/s eta 0:00:12
   --- ------------------------------------ 5.5/63.3 MB 5.4 MB/s eta 0:00:11
   --- ------------------------------------ 6.3/63.3 MB 5.6 MB/s eta 0:00:11
   ---- ----------------------------------- 6.8/63.3 MB 4.8 MB/s eta 0:00:12
   ---- ----------------------------------- 7.9/63.3 MB 4.8 MB/s eta 0:00:12
   ----- ---------------------------------- 8.9/63.3 MB 4.8 MB/s eta 0:00:12
   ------ --------------------------------- 10.2/63.3 MB 4.9 MB/s eta 0:00:11
   ------- -------------------------------- 11.5/63.3 MB 5.1 MB/s eta 0:00:11
   ------- -------

In [6]:
import cv2
import numpy as np

# Define paths for stereo images
left_image_path = "C:/Users/OmidTavasolifar/Downloads/Compressed/100831_155323_MultiCamera0_subset_db"
right_image_path = "C:/Users/OmidTavasolifar/Downloads/Compressed/100831_155323_MultiCamera0_subset_db"

# Stereo matching parameters
stereo = cv2.StereoBM_create(numDisparities=64, blockSize=15)

# Intrinsic parameters (example values, replace with dataset values)
focal_length = 718.856
baseline = 0.573

# Create a function to compute depth map
def compute_depth_map(left_img, right_img):
    # Convert to grayscale
    left_gray = cv2.cvtColor(left_img, cv2.COLOR_BGR2GRAY)
    right_gray = cv2.cvtColor(right_img, cv2.COLOR_BGR2GRAY)
    
    # Compute disparity
    disparity = stereo.compute(left_gray, right_gray).astype(np.float32) / 16.0
    
    # Avoid division by zero
    disparity[disparity <= 0] = 0.1
    
    # Calculate depth
    depth = (focal_length * baseline) / disparity
    return depth

# Process all image pairs
depth_maps = []
for i in range(100):
    left_img = cv2.imread(f"{left_image_path}/left_{i:03d}.png")
    right_img = cv2.imread(f"{right_image_path}/right_{i:03d}.png")
    depth = compute_depth_map(left_img, right_img)
    depth_maps.append(depth)

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [3]:
# Write depth maps to a video file
video_path = "depth_video.avi"
frame_size = (depth_maps[0].shape[1], depth_maps[0].shape[0])
out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'XVID'), 10, frame_size, False)

for depth in depth_maps:
    normalized_depth = cv2.normalize(depth, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    out.write(normalized_depth)

out.release()
print(f"Depth video saved to {video_path}")


IndexError: list index out of range

In [4]:
import open3d as o3d

def depth_to_point_cloud(depth_map, intrinsic_matrix):
    # Create meshgrid
    h, w = depth_map.shape
    u, v = np.meshgrid(np.arange(w), np.arange(h))
    
    # Unproject depth map to 3D points
    z = depth_map.flatten()
    x = (u.flatten() - intrinsic_matrix[0, 2]) * z / intrinsic_matrix[0, 0]
    y = (v.flatten() - intrinsic_matrix[1, 2]) * z / intrinsic_matrix[1, 1]
    
    points = np.vstack((x, y, z)).T
    return points

# Intrinsic matrix example (replace with dataset values)
intrinsic_matrix = np.array([[718.856, 0, 607.1928],
                             [0, 718.856, 185.2157],
                             [0, 0, 1]])

point_clouds = []
for depth_map in depth_maps:
    points = depth_to_point_cloud(depth_map, intrinsic_matrix)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    point_clouds.append(pcd)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
# Align point clouds using ICP
aligned_point_clouds = [point_clouds[0]]
current_pose = np.eye(4)

for i in range(1, len(point_clouds)):
    source = point_clouds[i]
    target = aligned_point_clouds[-1]
    
    # Apply ICP
    icp_result = o3d.pipelines.registration.registration_icp(
        source, target, 0.02,
        current_pose,
        o3d.pipelines.registration.TransformationEstimationPointToPoint()
    )
    
    # Transform point cloud
    transformed_pcd = source.transform(icp_result.transformation)
    aligned_point_clouds.append(transformed_pcd)
    current_pose = icp_result.transformation

# Combine aligned point clouds for visualization
final_pcd = o3d.geometry.PointCloud()
for pcd in aligned_point_clouds:
    final_pcd += pcd

# Visualize final point cloud
o3d.visualization.draw_geometries([final_pcd])

IndexError: list index out of range